### PydanticOutputParser

주요 메서드

- `get_format_instructions()`
- `parse()`

In [16]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
from langchain_teddynote import logging

logging.langsmith("edu-CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
edu-CH03-OutputParser


In [18]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

이메일 본문 예시 

In [20]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

출력 파서를 사용하지 않는 경우 예시

In [21]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

# prompt 내의 {} 값은 input_variable로 필수 입력 값
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션 소속이며, 이은채 대리에게 이메일을 보냈습니다.
2. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 활동하는 기업입니다.
3. 김철수 상무는 "ZENESIS" 자전거 모델에 대한 상세한 브로슈어를 요청하고 있습니다. 특히 기술 사양, 배터리 성능, 디자인 측면의 정보가 필요합니다.
4. 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안하고 있습니다.
5. 미팅 장소는 이은채 대리가 근무하는 회사의 사무실입니다.

In [22]:
print(output)

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션 소속이며, 이은채 대리에게 이메일을 보냈습니다.
2. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 활동하는 기업입니다.
3. 김철수 상무는 "ZENESIS" 자전거 모델에 대한 상세한 브로슈어를 요청하고 있습니다. 특히 기술 사양, 배터리 성능, 디자인 측면의 정보가 필요합니다.
4. 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안하고 있습니다.
5. 미팅 장소는 이은채 대리가 근무하는 회사의 사무실입니다.


Field 안의 `description` 은 특정 정보 추출 

In [23]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일은 보낸 사람")
    email: str = Field(description="메일은 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
pydantic_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [24]:
# instruction 추출
print(pydantic_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일은 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일은 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


프롬프트 정의 
1. `question` : 사용자 질문
2. `email_conversation` : 이메일 본문 내용
3. `format`: 형식 지정

In [25]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=pydantic_parser.get_format_instructions())

Chain 생성 및 실행

In [26]:
chain = prompt | llm

response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력
output = stream_response(response, return_output=True)

```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 유통 협력을 제안하며, 기술 사양, 배터리 성능, 디자인 정보가 포함된 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 화요일 오전 10시"
}
```

In [27]:
type(output)

str

`parser`를 사용하여 output 문자열을 EmailSummary 객체로 변환

In [28]:
# PydanticOutputParser 를 사용하여 결과 파싱
structured_output = pydantic_parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 유통 협력을 제안하며, 기술 사양, 배터리 성능, 디자인 정보가 포함된 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.' date='1월 15일 화요일 오전 10시'


### parser 가 추가된 체인 생성

In [ ]:
# 출력 파서를 추가하여 전체 체인을 구성
chain = prompt | llm | pydantic_parser

response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 유통 협력을 제안하며, 기술 사양, 배터리 성능, 디자인 정보가 포함된 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.', date='1월 15일 화요일 오전 10시')

In [37]:
response.summary

'김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 유통 협력을 제안하며, 기술 사양, 배터리 성능, 디자인 정보가 포함된 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.'

### Chain에 구조화된 출력 추가 

`.with_structured_output(Pydantic)`

In [38]:
llm_with_structured = ChatOpenAI(
    model_name="gpt-4o", temperature=0
).with_structured_output(EmailSummary)

In [39]:
answer = llm_with_structured.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 이은채 대리에게 바이크코퍼레이션과 테디인터내셔널 간의 협력 가능성을 논의하기 위해 연락했습니다. 김철수는 테디인터내셔널의 새로운 자전거 모델 "ZENESIS"에 관심을 표명하며, 기술 사양, 배터리 성능, 디자인에 대한 상세한 정보를 요청했습니다. 또한, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.', date='1월 15일 오전 10시')